In [2]:
# Standard Imports
from __future__ import print_function
import numpy as np
import pandas as pd
import os
import sys
from collections import defaultdict
from importlib import reload
from bs4 import BeautifulSoup
import requests
import scipy.stats as scs
import time

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Modeling
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import keras.backend as K
from tensorflow.keras.constraints import min_max_norm, non_neg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load MongoDB
from pymongo import MongoClient
client = MongoClient()
# Access/Initiate Database
db = client['producer_db']
# Access/Initiate Table
mfcc_tab = db['mfcc']
mfcc_collection = db.mfcc_tab

# Authorize Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Audio Processing
import librosa
import librosa.display
from IPython.display import Audio
from scipy.io import wavfile
from pydub import AudioSegment
from src.audio_processing import load_mp3_from_url

Using TensorFlow backend.


In [3]:
model = models.Sequential()

# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(12, 80, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# # model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten(input_shape = (20,1292,1)))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25840)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                258410    
Total params: 258,410
Trainable params: 258,410
Non-trainable params: 0
_________________________________________________________________


In [5]:
def mfcc_train_test(collection, test_size = 0.25, random_state = 440):
    """
    Inputs:
        collection: Mongo DB collection
        test_size: between 0 and 1, fraction of data in test set.
        random_state: random state for sklearn train_test_split
        
    Outputs:
        X_train, X_test, y_train, y_test, y_columns
        y_columns is the labels associated with the columns of y
    """
    X = []
    y = []
                    
    for song in collection.find():
        if np.array(song['MFCC']).shape[1] >= 1292:
            X.append(np.array(song['MFCC'])[:,:1292].flatten())
            y.append(song['producer'])
        
    
    #NEED TO STACK LIST OF MFCCS 
    X = np.vstack(X)
    y = np.array(y)
    
    y_one_hot = pd.get_dummies(y).values
    y_columns = pd.get_dummies(y).columns
#     X = X.reshape(-1,20,1292,1)
    
    X_train, X_test, y_train, y_test = train_test_split(X ,y_one_hot, test_size = test_size, random_state = random_state)
    
    return X_train, X_test, y_train, y_test, y_columns

In [6]:
X_train, X_test, y_train, y_test, y_columns = mfcc_train_test(mfcc_collection, test_size = 0.25, random_state = 440)

In [7]:
X_train.shape

(745, 25840)

In [8]:
# Standardize
ss = StandardScaler()
X_train_scale = ss.fit_transform(X_train)
X_test_scale = ss.transform(X_test)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scale.reshape(-1,20,1292,1), y_train, epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
